In [1]:
var ds = require('dstools');
var input = require('../examples/data/3features/input-m.json'); 
var kreativesFeld = require('../index');

#### Wie schneiden die beiden Algorithmen bei 100x Tests ab?  

Gruppengröße: 26  
Gruppenanzahl: 3

In [2]:
var groups = 3;

In [6]:
// 100x runs of monte carlo
var scoresMonteCarlo = [];
var resultsMonteCarlo = {};

// Initialize empty promise collection
var runs = [];

for(let run = 0; run < 100; run++) {
    
    // Calculate groups with monte carlo
    runs.push(kreativesFeld.monteCarlo(input, groups).then(result => {
                
        // Collect result and score
        resultsMonteCarlo[result.score] = result;
        scoresMonteCarlo.push(result.score);
        
        return;
    }));
}

var highscoreMonteCarlo = undefined;

// Run async tasks in a sequence
runs.reduce((promiseChain, currentTask) => {    
    return promiseChain.then(chainResults =>        
             currentTask.then(currentResult =>            
                  [ ...chainResults, currentResult ]        
             )    
        );
    },Promise.resolve([])).then(arrayOfResults => {    

        // Do something with all results
        console.log(scoresMonteCarlo);

        // Calculate highest score of this batch
        highscoreMonteCarlo = Math.max(...scoresMonteCarlo);

        console.log(highscoreMonteCarlo);
    });

[
  220.70948791228966, 233.02127095675343, 223.17015782876686, 218.13918280995895,
  216.82815878220606,  225.0618334770962, 223.10334955951504, 221.92219682342756,
   224.9035601116243, 239.10687362468735, 216.47214935268886, 226.04923423067692,
  227.53079127800243, 218.54398136264575, 216.63533575291268, 208.45237651511567,
   214.6947793282846, 216.41277209121307, 210.99241791791457, 219.18128544520206,
  225.05917473469438, 231.30284274898506, 234.06395478606814, 210.10271815932083,
   213.2444091952555, 223.69134975064475, 216.00491922745726, 205.67723289234877,
   213.6440439600721, 230.81660092195668, 215.47489016791133, 231.19797194233507,
  219.39254219118808, 215.17519216499184, 215.82367938903064, 215.14529789997337,
  220.60859113596177, 225.29237843404312, 222.56751279935523, 227.41753540854484,
   239.3257517088171, 227.36205089959347, 224.65442816119491,  218.6669721180121,
  219.51782137427972, 219.22182050998452, 235.86427791516599,  231.1554065032835,
  226.98732162

In [4]:
// 100x runs of genetic
var scoresGenetic = [];
var resultsGenetic = {};
var runs = [];

// Run algorithm x times
for(let run = 0; run < 100; run++) {
    runs.push(kreativesFeld.genetic(input, groups).then(result => {
        
        // Collect result and score
        resultsGenetic[result.score] = result;
        return scoresGenetic.push(result.score);
    }));
}

var highscoreGenetic = undefined;

// We must wait until all calculations are done
Promise.all(runs).then(values => {
    
    console.log(scoresGenetic);

    // Calculate highest score of this batch
    highscoreGenetic = Math.max(...scoresGenetic);

    console.log(highscoreGenetic);
});

[
  206.63658706495772, 194.07623287831314, 214.85039664833783,  209.8507519311126,
  216.06687530702075,  231.5068801644299, 221.22691641993376,  221.0159460858159,
  201.37730161387853, 211.63884572940384, 209.69601247083912, 212.45051903155405,
   203.0574735934613, 201.13558742076987, 204.28547670539558, 218.69200356493022,
    218.350581013456, 206.33651053062448,  215.2546913517066,  211.6367475829017,
  231.49074868887473, 206.93369300200507, 190.65664480814988,  218.7065455628805,
  227.13770205682815,   217.905518740106, 212.72974605520432, 202.17363621885335,
  212.78725193066558,  209.2111304380516, 204.92276587557825,  219.1620091529531,
  205.01806768370113,   205.492227008882, 213.00490698188858, 219.49945961180387,
   218.8763619096424,  220.9514156062428,   225.041352321602,  227.5050089770769,
   219.7297704209889, 221.40106754224402, 218.58730007603577,  211.7378949387135,
  215.00364202213478, 236.09187726481733, 226.93068124008917, 228.74750829452475,
  198.15588137

## Vergleich Algorithmen

In [7]:
// Calculate overall hitrate

// Highest score
var highestScore = (highscoreMonteCarlo > highscoreGenetic) ? highscoreMonteCarlo : highscoreGenetic;

var hitRateMonteCarlo = scoresMonteCarlo.map(v => v/highestScore);
var hitRateGenetic = scoresGenetic.map(v => v/highestScore);

In [8]:
var trace1 = {
  x: hitRateMonteCarlo,
  type: "histogram",
  opacity: 0.5,
  marker: {
     color: 'green',
  },
  name: 'Monte Carlo'
};
var trace2 = {
  x: hitRateGenetic,
  type: "histogram",
  opacity: 0.6,
  marker: {
     color: 'red',
  },
  name: 'Genetic Algorithm'
};

var data = [trace1, trace2];
var layout = {barmode: "overlay", xaxis: {title: 'Trefferquote'}, yaxis: {title: 'Häufigkeit'}};

ds.Collection().plotly(data, layout).show();

## Darstellung "bestes" Ergebnis

In [9]:
// Retrieve winner combination
var winnerSet = (highestScore == highscoreMonteCarlo) ? resultsMonteCarlo : resultsGenetic;
var winner = winnerSet[highestScore];

// Generate empty traces
/*var traces = ['rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)'].map(c => {
    return {x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
            color: c,
            width: 0.5},
            opacity: 0.8},
        type: 'scatter3d'};
});
*/
var traces = [];
for(let i = 0; i < groups; i++) {
    traces.push({x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
                color: 'rgba(217, 217, 217, 0.14)',
                width: 0.5
            },
            opacity: 0.8
        },
        type: 'scatter3d'
    });
}

// Loop input data and build group traces
Object.keys(input).forEach((member, index) => {
    
    // Retrieve member group 
    var group = winner.combination[index];
    
    traces[group-1].x.push(input[member][0]);
    traces[group-1].y.push(input[member][1]);
    traces[group-1].z.push(input[member][2]);    
});

// Print 3D Model with group traces and colors
ds.Collection().plotly(traces,{margin: {l: 0, r: 0, b: 0, t: 0 }}).show();